# <center>Curso de Modelagem de Dados para IA - PARTE 5</center>

<img src="img/image.jpg" alt="Drawing" style="width: 300px;"/>


## Lendo Dados com SQL
Muitos conjuntos de dados úteis e importantes são armazenados em bancos de dados relacionais que suportam a Linguagem de Consulta Estruturada (SQL). Bancos de dados relacionais (DBs) são essencialmente coleções de tabelas, semelhantes a dataframes ou planilhas, que contêm relacionamentos entre tabelas na forma de chaves compartilhadas (identificadores de diferentes itens de dados). A ligação entre tabelas por meio dessas chaves compartilhadas permite construir estruturas de dados mais complexas além das duas dimensões definidas por uma única tabela. Neste tutorial, pretendemos principalmente fornecer algumas orientações sobre como se conectar a bancos de dados SQL em Python e converter alguns dos resultados de consultas SQL em estruturas de dados Python, como Pandas DataFrames, para processamento adicional.

Existem muitas ferramentas diferentes que gerenciam bancos de dados relacionais, denominadas – apropriadamente – Sistemas de Gerenciamento de Banco de Dados Relacionais (SGBD). Isso inclui grandes aplicativos que geralmente residem em máquinas remotas dedicadas a fornecer acesso a um banco de dados — como Oracle, MySQL e Microsoft SQL Server — bem como SGBDs mais leves que você pode instalar e executar em seu próprio desktop/laptop, como SQLite e SmallSQL. Cada um apresenta uma sintaxe um pouco diferente para se conectar a eles a partir de um ambiente externo, como Python, e uma variedade de bibliotecas Python foram criadas para oferecer suporte a essas interações. A maioria desses sistemas, se não todos, envolve a criação de um objeto dentro do Python que representa uma conexão com um mecanismo SGBD, que pode ser usado para ler/gravar dados de/para o banco de dados. A biblioteca **SQLAlchemy** é um Toolkit **Python para SQL e mapeamento objeto relacional** que fornece uma variedade de operações para trabalhar com bancos de dados SQL e visa fornecer — entre muitas outras coisas — uma interface comum para conexão a uma variedade de SGBDs. Além disso, a biblioteca Pandas discutida anteriormente é capaz de ler e gravar dados não apenas de arquivos simples e planilhas, mas também de um objeto de conexão SQLAlchemy. Vamos demonstrar um pouco dessa funcionalidade aqui.

A peça chave que usaremos do SQLAlchemy é a função *create_engine*, que cria um objeto de mecanismo de banco de dados que podemos ler ou gravar usando instruções SQL. Na maioria das vezes, essa função pode ser chamada com um único argumento — uma string de conexão — que descreve como se conectar ao SGBD e ao banco de dados de interesse. Cada SGBD tem sua própria sintaxe para configurar tais conexões, e obter a string de conexão correta às vezes pode exigir alguma experimentação, mas a página de configuração do mecanismo SQLAlchemy fornece muitos exemplos. Neste tutorial, estamos principalmente interessados em criar e, posteriormente, ler de um banco de dados SQLite em nosso disco local. Em particular, como os dados do Dataset sobre Beisebol consistem em várias tabelas inter-relacionadas, ele fornece um exemplo de aplicação útil para essa funcionalidade.

### A interoperabilidade entre SQLAlchemy e Pandas
A instrução a seguir abre uma conexão SQLAlchemy com um banco de dados SQLite armazenado em um arquivo chamado *mydb.sqlite*:

In [6]:
from sqlalchemy import create_engine
import glob, os
import pandas as pd

In [7]:
def read_all_databank_core_csv(directory):
    """
    read all csv files in the specified baseball databank directory and
    populate a dictionary storing each of the tables keyed to its name
    """
    dfs = {}
    files = glob.glob('{}/*.csv'.format(directory))
    for f in files:
        d, name = os.path.split(f)
        table = os.path.splitext(name)[0]
        df = pd.read_csv(f)
        dfs[table] = df
    return dfs

bbdfs = read_all_databank_core_csv('data/baseballdatabank/core')

In [8]:
engine = create_engine('sqlite:///mydb.sqlite')

Lembre-se de que os dados de beisebol foram armazenados em um dicionário Python contendo 27 dataframes, cada um com seu respectivo nome. Usando o *to_sql* definido nos dataframes do Pandas, podemos escrever cada uma dessas 27 tabelas em um banco de dados sqlite. A função a seguir cria um mecanismo, faz um loop sobre cada um dos dataframes armazenados no dicionário *dfs* e o grava no banco de dados e, em seguida, descarta o mecanismo no final:

In [9]:
def write_all_tables_to_sqlite(dfs, sql_filename):
    engine = create_engine('sqlite:///{}'.format(sql_filename))
    for table, df in dfs.items():
        df.to_sql(table, con=engine, index=False)
    engine.dispose()

E o código a seguir chamará essa função para gravar todos os dataframes em um banco de dados chamado bbdb.sqlite:

In [10]:
sqlite_filename = 'bbdb.sqlite'
try:
    os.remove(sqlite_filename)
except FileNotFoundError:
    pass

write_all_tables_to_sqlite(bbdfs, sqlite_filename)

Com todos os dados em um banco de dados SQLite, agora podemos nos conectar a ele e emitir consultas SQL para extrair informações de interesse. Felizmente, assim como o pandas fornece uma função conveniente para gravar dataframes em um banco de dados, ele também fornece uma função útil para executar uma consulta SQL e retornar os resultados armazenados em um dataframe. Não abordaremos a sintaxe das consultas SQL aqui, mas forneceremos um exemplo específico: código para selecionar - da tabela de rebatidas no mecanismo de banco de dados - os 10 principais acertos (H) produzidos por um rebatedor em uma única temporada:

In [13]:
engine = create_engine('sqlite:///bbdb.sqlite')
most_hits_in_a_year = pd.read_sql_query('select * from batting order by H desc limit 10', engine)

In [14]:
most_hits_in_a_year

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,suzukic01,2004,1,SEA,AL,161,704,101,262,24,...,60.0,36.0,11.0,49,63.0,19.0,4.0,2.0,3.0,6.0
1,sislege01,1920,1,SLA,AL,154,631,137,257,49,...,122.0,42.0,17.0,46,19.0,NaN,2.0,13.0,NaN,NaN
2,odoulle01,1929,1,PHI,NL,154,638,152,254,35,...,122.0,2.0,NaN,76,19.0,NaN,4.0,13.0,NaN,NaN
3,terrybi01,1930,1,NY1,NL,154,633,139,254,39,...,129.0,8.0,NaN,57,33.0,NaN,1.0,19.0,NaN,NaN
4,simmoal01,1925,1,PHA,AL,153,654,122,253,43,...,129.0,7.0,14.0,35,41.0,NaN,1.0,6.0,NaN,NaN
5,hornsro01,1922,1,SLN,NL,154,623,141,250,46,...,152.0,17.0,12.0,65,50.0,NaN,1.0,15.0,NaN,NaN
6,kleinch01,1930,1,PHI,NL,156,648,158,250,59,...,170.0,4.0,NaN,54,50.0,NaN,4.0,13.0,NaN,NaN
7,cobbty01,1911,1,DET,AL,146,591,147,248,47,...,127.0,83.0,NaN,44,NaN,NaN,8.0,11.0,NaN,NaN
8,sislege01,1922,1,SLA,AL,142,586,134,246,42,...,105.0,51.0,19.0,49,14.0,NaN,3.0,16.0,NaN,NaN
9,suzukic01,2001,1,SEA,AL,157,692,127,242,34,...,69.0,56.0,14.0,30,53.0,10.0,8.0,4.0,4.0,3.0


Para quem é fã de beisebol, talvez já saiba que Ichiro Suzuki (playerID suzukic01) atualmente detém o recorde de mais rebatidas em uma temporada (262 rebatidas em 2004), tendo superado o recorde de longa data de George Sisler (257 rebatidas em 1920). Como não é o caso da grande maioria, iremos normalmente procurar pelos nomes anexados aos playerIDs, um tanto obscuros, armazenados na tabela de rebatidas. Para isso emitimos uma consulta SQL mais complicada usando uma junção de tabela para extrair os nomes dos jogadores da tabela de pessoas (usando as aspas triplas do Python para criar um string de várias linhas para acomodar a consulta longa):

In [15]:
pd.read_sql_query("""select people.nameFirst, people.nameLast, batting.playerID, 
batting.yearID, batting.H from people join batting on people.playerID=batting.playerID 
order by batting.H desc limit 10""", engine)

,nameFirst,nameLast,playerID,yearID,H
0,Ichiro,Suzuki,suzukic01,2004,262
1,George,Sisler,sislege01,1920,257
2,Lefty,O'Doul,odoulle01,1929,254
3,Bill,Terry,terrybi01,1930,254
4,Al,Simmons,simmoal01,1925,253
5,Rogers,Hornsby,hornsro01,1922,250
6,Chuck,Klein,kleinch01,1930,250
7,Ty,Cobb,cobbty01,1911,248
8,George,Sisler,sislege01,1922,246
9,Ichiro,Suzuki,suzukic01,2001,242


### Qual é o sentido de todo esse vai-e-vem entre pandas e SQL?
Este exemplo pode parecer um pouco artificial. Para começar, os dados estavam em um formato csv padrão, que pudemos ler convenientemente nos dataframes do pandas para aproveitar todo o poder de processamento que o pandas fornece. Mas então escrevemos todos esses dataframes em um banco de dados SQLite, apenas para que pudéssemos lê-los de volta em pandas, o que requer saber como criar consultas SQL. Embora reconhecidamente um fluxo de trabalho um tanto atípico, aqui estão pelo menos algumas razões diferentes pelas quais pode ser útil usar algum desse tipo de funcionalidade em seu próprio código:

- Os dados já estejam armazenados em um banco de dados SQL e precisamos extraí-los para poder processá-los no pandas.

- A familiaridade com o SQL pode tornar mais confortável para o desenvolvedor extrair informações direcionadas usando essa sintaxe em vez de descobrir como fazer algo análogo em pandas. Apesar disso, o Pandas fornece funcionalidades substanciais para mesclar, unir e concatenar dataframes de uma maneira semelhante aos tipos de operações de junção fornecidas pelo SQL.

- A quantidade de dados é muito grande e será necessário somente extrair subconjuntos deles para cálculos específicos. Em vez de ter que manter todos esses dados na memória (o que pode nem ser possível), pode-se usar consultas SQL para extrair apenas as partes com as quais precisa trabalhar mais.